In [ ]:
# !git clone https://github.com/dsa-playground/tvt2024.git
# %cd /content/tvt2024/
# !git pull
# !pip install upgrade -r requirements.txt -t "tvt2024"
# !pip install pyaml-env

In [ ]:
# Imports
import datetime
import warnings
import pandas as pd
from scripts.main import laad_data, bekijk_ziekteverzuim, bekijk_clienten, bekijk_flexpool, kies_onderwerp, pas_voortschrijdend_gemiddelde_toe, pas_regressie_toe, onderzoek_afwijkingen, bereken_metrieken, voorspel, optie_1, optie_2, optie_3

## Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)
datum_vandaag = datetime.datetime.now().strftime("%Y-%m-%d")
# Disable warnings
warnings.filterwarnings('ignore')

# 1. Inleiding

Jullie zijn allen werkzaam bij zorgorganisatie 'Zorgello' in de afdeling Analytics. De bestuurder komt binnen stormen:

Morgen heb ik een vergadering met de Raad van Commissarissen. Vorig jaar hebben ze me flink aan de tand gevoeld. Van een aantal zaken kon ik niet goed aangeven welke richting het op beweegt. Ik heb voor de vergadering morgen wat forecasts nodig. Kunnen jullie deze even snel maken? Het gaat om:
1. Het aantal cliënten (Zorgzwaartepakket 6 of hoger)
2. Het ziekteverzuimpercentage van ons personeel
3. Inzet van flexpool medewerkers

Jullie als afdeling verdelen de onderwerpen. 

Bekijk de data van de verschillende reeksen.

In [ ]:
data = laad_data()
bekijk_clienten(data)

Nader te bepalen tekst bij clienten

In [ ]:
bekijk_ziekteverzuim(data)

Nader te bepalen tekst bij ziekteverzuim

In [ ]:
bekijk_flexpool(data)

Nader te bepalen tekst bij inzet flexpool (personeel)

Wanneer dezelfde variabele op verschillende momenten gemeten worden ontstaat een tijdreeks. In tijdreeksen wordt vaak onderscheid gemaakt tussen:
* Trend: richting (stijgend/dalend)
* Seizoenspatroon/-patronen: terugkerend patroon (bepaalde momenten standaard hoger/lager)
* Ruis: afwijkingen niet te verklaren door trend of patroon

Afhankelijk van welke tijdreeks je bekijkt zijn er 1 of meerdere van de bovenstaande onderdelen aanwezig. 

**Opgave 1.1.** Een bekende tijdreeks is de temperatuur.

a. Wat weten we van de trend in temperatuur?

<!-- Temperatuur neemt toe door invloed mens: Stijgende trend. -->

b. Welke seizoen patronen kun je herkennen in de temperatuur?

<!-- - Dag patroon ('s nachts kouder dan overdag, invloed zonopwarming)
- Jaarpatroon (4 seizoenen, draaiing aarde rond de zon) -->

c. Wat zijn mogelijke factoren die ruis veroorzaken?

<!-- Bewolking, wind- en waterstromingen, zonnevlammen, smog/uitstoot, ... -->

**Opgave 1.2.** Kies een van de tijdreeksen om te voorspellen voor de bestuurder. Door de code in de onderstaande cel te runnen (druk op 'play knop') krijg je de vraag en een invoerveld.  


In [ ]:
onderwerp = kies_onderwerp()

# 2. Modellen

Nu je het onderwerp gekozen hebt, kunnen we een aantal modellen gaan toepassen. 

Een voorspelling is een schatting voor de toekomst. Hoe meer historische, proces en causale informatie, hoe beter een algoritme te bouwen is voor een voorspelling. Om in deze workshop niet te verzanden in verklarende data en complexe modellen, hebben we gekozen voor twee ('eenvoudige') modellen:
* Het voortschrijdend gemiddelde
* Het regressiemodel


#### *Het voortschrijdend gemiddelde*
Het voortschrijdend gemiddelde maakt gebruik van de historische waarden van het onderwerp. Op basis van een aantal voorgaande waarden wordt een gemiddelde berekend. Dit gemiddelde is de voorspeller voor de toekomst. 

Voor de volledigheid en wiskunde liefhebbers, de formule luidt:

$$
\text{Voortschrijdend gemiddelde} = \frac{\left( a_{n-k+1} + a_{n-k+2} + ... +a_{n}\right)}{k} = \frac{1}{k} \sum_{i=n-k+1}^n a_i
$$

Hierbij is 'a' de historische waarde, 'n' geeft de volgorde van de historische waarde aan en k het aantal voorgaande waarden. Het voortschrijdend gemiddelde is het gemiddelde van de laatste *k* waarden. 

**Opgave 2.1.** Stel het voortschrijdend gemiddelde wordt bepaald op basis van de 4 voorgaande waarden. Hoe zien de eerste 2 getallen (a en b) eruit na deze reeks:

*9, 16, 12, 8, 12, a, b*

a = ...

b = ...


<!-- a = (16 + 12 + 8 + 12) / 4 = 12

b = (12 + 8 + 12 + 12) / 4 = 11 -->


Laten we het voortschrijdend gemiddelde toepassen voor het onderwerp wat jullie gekozen hebben. 


In [ ]:
voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(data=data, onderwerp=onderwerp)

**Opgave 2.2.** Wat valt je op aan de uitkomsten van het voortschrijdend gemiddelde model?

<!-- - 1e week in de traindata heeft geen waarde.
- Na een periode van voorspellingen onstaat een constante waarde.  -->


#### *Regressie*
Er zijn ook algoritmes welke niet enkel kijken naar de historische waarden die je probeert te voorspellen. Deze algoritmes maken gebruik van verklarende variabele(n). Het eenvoudigste model is een regressiemodel. Wat dit model doet een regressielijn bepalen: best passende lijn om de relatie tussen verklarende variabelen (x) en te voorspellen variabelen (y) te bepalen. 

Wederom voor de wiskunde liefhebbers, in formulevorm:

$$
y_i = \beta_0 + \beta_1 x_1 + ... + \beta_i x_i + \epsilon_i
$$

Laten we een eenvoudige regressie met 1 variabele (volgordelijkheid datums) toepassen:

In [ ]:
regressie = pas_regressie_toe(data=data, onderwerp=onderwerp)

**Opgave 2.3.** Welke verklarende variabele heeft deze tijdreeks?

<!-- Datumreeks / dag -> deze wordt onder water numeriek gemaakt. -->

# 3. Evalueren

Elk van de modellen geven een voorspelling. Natuurlijk is een visuele check (realisatie naast voorspelling) heel erg belangrijk. Toch is het ook belangrijk om iets meer te kijken naar de afwijking op zichzelf. 

#### *De afwijking (error)*
Laten we de reeks van errors voor elk model eens bekijken voor de testperiode.  

In [ ]:
onderzoek_afwijkingen(list_of_dfs=[voortschrijdend_gemiddelde, regressie], 
                      onderwerp=onderwerp)

Bij de toegepaste modellen wordt voor de train periode natuurlijk een 'optimum' gezocht binnen de grenzen van een model. Maar afhankelijk van het onderwerp en het model zul je verschillen zien.

**Opgave 3.1.** Welk model heeft de grootste afwijking?

<!-- Dit is een instinker. De grootte van een afwijking wordt vooral bepaald door wat iemand belangrijk vindt. Temperatuur voorspellen voor kledingkeuze is nog te overzien. Aankopen op de beurs doen kan financieel grote gevolgen hebben. Het gaat dus om hoe nauwkeurig de voorspelling moet zijn op het vervolgproces zo foutloos mogelijk te doen.  -->

**Opgave 3.2.**
Naar welke punten kun je (visueel) kijken bij het beoordelen van de tijdreeks van errors?

<!-- a. Ligt de error rond een bepaalde vaste afwijking? 
b. Neemt de grootte van de afwijking af/toe over tijd? Dit noemen we 'modeldrift'
c. Zijn er extremen te herkennen? Als de error grote afwijkingen kent, dan is er in de oorspronkelijke data heel veel ruis. De vraag is of deze ruis te corrigeren is (in verleden en toekomst). Zo niet, dan moet je je afvragen welke afwijking hinderlijk zijn in het acteren op een voorspelling.  -->

#### *Metrieken*
Om subjectiviteit eruit te halen is het belangrijk stil te staan bij het selecteren van een goede metriek. Een metriek is een getalsmatige uitdrukking van hoe goed/slecht je model presteert. Er zijn ontzettend veel metrieken, elk met een specifiek inzicht en doel. Het toepassen van meerdere metrieken voor de beoordeling van een model wordt ook aangeraden.

Voor deze workshop hebben we 4 metrieken gekozen welke inzicht geven in de prestatie van de modellen:
* **Juistheid**: Deze geeft aan hoe goed de voorspelling is voor de gehele testperiode. 1 is gelijk, kleiner dan 1 is te laag en groter dan 1 is te hoog. 
* **Maximale afwijking**: Voor elke dag is er een voorspelling en een realisatie in de testperiode. Deze metriek geeft de maximale afwijking weer. Hiermee zou je de maximale 'schade' kunnen bepalen als een voorspelling gebruikt wordt voor een vervolg(proces).
* **Gemiddelde absolute afwijking**: Deze wordt berekend met Mean Absolute Error. Deze metriek berekend het gemiddelde van de verschillen in absolute waarde (alles wordt positief getal). Het geeft daarmee inzicht in hoe groot de gemiddelde afwijking echt is. Hoe dichter bij 0, hoe beter het model presteert. 




In [ ]:
bereken_metrieken(list_of_dfs=[voortschrijdend_gemiddelde, regressie], 
                  onderwerp=onderwerp)

**Opgave 3.3.** Bij meerdere metrieken kan het voorkomen dat voor sommige metrieken model A beter presenteert en voor andere metrieken model B. Hoe bepaal je dan welk model je moet kiezen? Wat is belangrijk voor jouw gekozen onderwerp?

<!-- Je kiest metrieken die aansluiten bij je behoefte. Daarnaast is het belangrijk om een volgorde toe te kennen welke metriek je het belangrijkste vindt en hoe groot een afwijking doorslaggevend is. Ook hierbij: Wat is de impact in een vervolgproces!  -->

# 4. Verbeteren model

In de bovenstaande voorbeelden zijn standaard instellingen gekozen. Door het aanpassen van de instellingen, kun je tot een beter model komen. 

**Instellingen voor het voortschrijdend gemiddelde**
Het voortschrijdend gemiddelde kent in deze toepassing twee variabelen:
* *vensterlengte*: Het aantal dagen voorafgaand waar het gemiddelde voor berekend wordt. Oftewel, bij 7 wordt het gemiddelde genomen van 7 voorafgaande dagen (lees: week). De minimumwaarde voor vensterlengte is 1. 
* *verschuiving*: Het aantal dagen dat teruggekeken moet worden. Oftewel, bij 28 (4-wekelijks patroon) gaat het model 4 weken terug en neemt dan de voorgaande window_size om het gemiddelde te bepalen. De minimumwaarde voor verschuiving is 0.

Zie tevens het plaatje hieronder:

![Instellingen bij voortschrijdend gemiddelde](https://raw.githubusercontent.com/dsa-playground/tvt2024/main/images/voortschrijdend_gem_instellingen.png)

**Instellingen voor regressie**
Het regressiemodel kent in deze toepassing drie variabelen:
* *jaarlijks_patroon*: Via deze variabele geef je aan of je een jaarlijks seizoenspatroon mee wil nemen. Hierdoor ontstaan extra variabelen die datum omzetten in een jaarpatroon. Om dit effect wel/niet mee te nemen kies je voor True/False. 
* *wekelijks_patroon*: Via deze variabele geef je aan of je een weeklijks seizoenspatroon mee wil nemen. Hierdoor ontstaan extra variabelen die datum omzetten in een weekpatroon. Om dit effect wel/niet mee te nemen kies je voor True/False. 
* *graad*: De graad bepaald hoe de data omgezet wordt om een vergelijking te kunnen toepassen. Voor de ene vergelijking zijn meer variabelen nodig als anderen. Bij een graad van 1 krijg je een rechte lijn (y = ax+b), graad van 2 een kwadratische lijn (y = ax^2 + bx + c), etc. De minimumwaarde voor graad is 1. 

Zie tevens plaatje hieronder:

![Instellingen bij regressie](https://raw.githubusercontent.com/dsa-playground/tvt2024/main/images/regressie_instellingen.png)

**Opgave 4.1.** Geavanceerde vraag: Waarom is er geen maandelijks_patroon als instelling bij regressie?

<!-- Maanden zijn onderling niet te vergelijken. Het aantal dagen verschilt per maand en niet elke dag begint op dezelfde weekdag. Bij deze data is er geen onderliggend proces dat vraagt om een maandpatroon. -->

**Opgave 4.2.** In de onderstaande tabel staan verschillende instellingen. Optie 0 zijn de instellingen zoals hierboven gebruikt. Welke van de volgende instelling levert het beste resultaat op? 

|Optie 0 | Optie 1 | Optie 2 | Optie 3 |
|:---|:---|:---|:---|
|Voortschrijdend gemiddelde:<ul><li>vensterlengte = 7</li><li>verschuiving = 0</li></ul>|Voortschrijdend gemiddelde:<ul><li>vensterlengte = 1</li><li>verschuiving = 7</li></ul>|Voortschrijdend gemiddelde:<ul><li>vensterlengte = 1</li><li>verschuiving = 365</li></ul>|Voortschrijdend gemiddelde:<ul><li>vensterlengte = 7</li><li>verschuiving = 28</li></ul>|
|Regressiemode:<ul><li>jaarlijks_patroon=False</li><li>wekelijks_patroon=False</li><li>graad=1</li></ul>|Regressiemode:<ul><li>jaarlijks_patroon=True</li><li>wekelijks_patroon=False</li><li>graad=1</li></ul>|Regressiemode:<ul><li>jaarlijks_patroon=False</li><li>wekelijks_patroon=True</li><li>graad=3</li></ul>|Regressiemode:<ul><li>jaarlijks_patroon=True</li><li>wekelijks_patroon=True</li><li>graad=12</li></ul>|


In [ ]:
optie_1(data, onderwerp)

In [ ]:
optie_2(data, onderwerp)

In [ ]:
optie_3(data, onderwerp)

# Voorspel

Als je een model en de (beste) instellingen gevonden hebt, dan rest uiteraard het genereren van de voorspelling. 
Door het draaien van de onderstaande cel wordt een voorspelling gemaakt tot 1 januari 2026. 

In [ ]:
voorspelling = voorspel(
    data=data,
    onderwerp=onderwerp,
    voorspellen_tot_datum='2026-01-01'
)

Hierboven zie je een uitkomst met de beste instellingen. In opgave 4.2 heb je een aantal opties gekregen. Dit was vooral om wat zicht te krijgen op wat een instelling doet. Uiteraard wordt bij de ontwikkeling en toepassing vaak gebruik gemaakt van de rekenkracht van een computer om de optimale instellingen te vinden. 

**Opgave 4.3.** Bonusvraag: Kan jij al proberend het model en de instellingen vinden die horen bij deze metrieken voor het onderwerp wat je gekozen hebt? 

*Hint*: Probeer eerst met grotere wijzigingen een beeld te krijgen waar je moet zoeken. 

|Onderwerp|Juistheid|Maximale afwijking|Gemiddelde absolute afwijking|
|:---|:---|:---|:---|
|Cliënten|1,001107927|3,208861768|1,048138511|
|Ziekteverzuim|1,000165092|0,718964844|0,194702055|
|Flexpool|1,000232416|9,411949646|1,613120387|

* Model = .....?

* Voortschrijdend gemiddelde
    - vensterlengte = .....?
    - verschuiving = .....?

* Instellingen regressiemodel
    - jaarlijks_patroon = .....?
    - wekelijks_patroon = .....?
    - graad = .....?

In [ ]:
## Instellingen model voortschrijdend gemiddelde
vensterlengte = 7
verschuiving = 0

## Toepassen instellingen in model
df_voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(
    data=data,
    onderwerp=onderwerp,
    vensterlengte=vensterlengte,
    verschuiving=verschuiving,
    zie_traintest_periodes=True
)

## Berekenen metrieken
bereken_metrieken(list_of_dfs=[df_voortschrijdend_gemiddelde], 
                  onderwerp=onderwerp)

In [ ]:
## Instellingen regressiemodel
jaarlijks_patroon=False
wekelijks_patroon=False
graad = 1

## Toepassen instellingen in model
df_regressie = pas_regressie_toe(data=data,
    onderwerp=onderwerp,
    jaarlijks_seizoenspatroon=jaarlijks_patroon,
    wekelijks_seizoenspatroon=wekelijks_patroon,
    graad=graad,
    zie_traintest_periodes=True
)

## Berekenen metrieken
bereken_metrieken(list_of_dfs=[df_regressie], 
                  onderwerp=onderwerp)